In [800]:
# import relevant packages
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import csv
%matplotlib qt


In [801]:
def distribute(needs,space): #distributes cars, there might be a faster way, but i cannot think of one right now
    endresult=np.zeros(len(needs),dtype=int)
    # filter=needs
    while space>=np.sum((needs>0)):
        add=min(np.min(needs[needs>0]),space//np.sum((needs>0)))
        endresult[needs>0]+=add
        space-=add*np.sum((needs>0))
        needs[needs>0]-=add
    endresult[np.nonzero(needs>0)[0][:space]]+=1
    return endresult
class edge(object):
    def __init__(self,length,lanes,speed=100,id=0,delta=1):
        self.length = int(length)
        self.lanes = lanes
        self.speed=int(speed)
        self.cars = np.zeros((int(lanes*length/(4.5+55)),5)) #collums: occupied,age,remaining travel time, spawnpoint, destination
        self.id=id
        self.delta=delta
        self.leftthismin=0
        self.leftthismin_avg=0
    def __str__(self): 
        return f"edge {self.id} cars:{np.sum(self.cars[:,0])}"
    def __repr__(self): 
        return f"edge {self.id} cars:{np.sum(self.cars[:,0])}"
    def C(self): #capacity of edge
        return int(self.delta*self.lanes*self.length/(4.5+55))
    def tto(self):
        return self.length/self.speed*60/1000
    def update(self):
        Filter = np.nonzero(self.cars[:,0]==1)[0] #Filter for 
        self.cars[Filter,2] -= 1 #travelled one minute
        self.cars[Filter,1] += 1 #aged one minute
        self.leftthismin_avg=self.leftthismin_avg*9/10+self.leftthismin/10
        self.leftthismin=0
    def tt(self):
        return self.tto()*(1+alpha*(self.size()/self.C())**beta)
    def size(self):
        return sum(self.cars[:,0])
class node(object):
    def __init__(self,input=[],output=[],destinations=[],id=0):
        self.id=id
        self.output = output
        self.input = input
        self.destinations=destinations
        self.waitlist=dict(zip(self.output,[dict(zip(self.input,[np.zeros(0,dtype=int) for j in self.input]))for i in self.output]))
        self.buildup=np.zeros((0,5))
        self.times=np.array([])
    def initwaitlist(self):
        if len(self.waitlist.keys())>0: print("waitlist already had keys",self.waitlist); return
        else: self.waitlist=dict(zip(self.output,[dict(zip(self.input,[np.zeros(0,dtype=int) for j in self.input]))for i in self.output]))
    def __str__(self): 
        if len(self.buildup): return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]} and buildup:{len(self.buildup)}"
        else: return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]}"
    def __repr__(self): 
        if len(self.buildup): return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]} and buildup:{len(self.buildup)}"
        else: return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]}"
    def size(self):
        return len(self.buildup)
    def flow(self):
        for i in self.input:
            Remove=np.nonzero(np.logical_and(i.cars[:,4]==self.id,np.logical_and(i.cars[:,2]<=0,i.cars[:,0]==1)))
            self.times=np.concatenate((self.times,i.cars[Remove,1].flatten())) #remove all cars that have arrived at their destination
            i.cars[Remove,:]=0
            i.leftthismin+=len(Remove[0])

            if self.output==[]: continue
            Filter2 = np.nonzero(np.logical_and(i.cars[:,2]==0,i.cars[:,0]==1))[0] #check which cars arrived now
            p=np.ones(len(self.output))
            for index,edge in enumerate(self.output):
                if np.sum(edge.cars[:,0])==0: p[index]=1/edge.tto()
                else: p[index]=1/(np.sum(edge.cars[edge.cars[:,2]>0,2])/np.sum(edge.cars[:,0]))
            choices=np.random.choice(self.output,len(Filter2),p=p/np.sum(p)) ### can be replaced by larger function to better decide the choices
            for j in self.output:
                self.waitlist[j][i]=np.concatenate((self.waitlist[j][i],np.array(Filter2[choices==j],dtype=int))) #add new indexes along with their location at the end
        for i in self.output:
            if sum([len(self.waitlist[i][j]) for j in self.input])==0: continue #doesn't need to do anything if there are no cars waiting
            elif sum([len(self.waitlist[i][j]) for j in self.input])<=np.sum(i.cars[:,0]==0):
                for j in self.input:
                    filter=np.nonzero(i.cars[:,0]==0)[0][:len(self.waitlist[i][j])]
                    i.cars[filter]=j.cars[self.waitlist[i][j],:] #put cars in next edge
                    i.cars[filter,2]=np.ceil(i.tt()+np.random.normal(0,2,size=len(filter)))
                    j.cars[self.waitlist[i][j],:]=0 #empty the cars
                    j.leftthismin+=len(filter)
                    self.waitlist[i][j]=np.zeros(0,dtype=int) #empty waitlist
            else:
                space=np.sum(i.cars[:,0]==0)
                needs=np.array([len(self.waitlist[i][j]) for j in self.input])
                allowances=distribute(needs,space)
                for k,j in enumerate(self.input):
                    j.leftthismin+=allowances[k]
                    filter=np.nonzero(i.cars[:,0]==0)[0][:allowances[k]]
                    i.cars[filter]=j.cars[self.waitlist[i][j][:allowances[k]]] ###probs dont need [filtertje,:] [filtertje] seems enough
                    i.cars[filter,2]=np.ceil(i.tt()+np.random.normal(0,2,size=len(filter)))
                    j.cars[self.waitlist[i][j][:allowances[k]],:]=0 #empty the cars
                    self.waitlist[i][j]=self.waitlist[i][j][allowances[k]:]
    def generate(self,state,mean=85,sd=1.5,destinations=[2]): ##change code to account for more generating cities and differing distributions
        N=int(np.random.normal(mean,sd)+0.5) #+.5 rounds it to the nearest integer
        p=np.ones(len(self.output))
        for index,edge in enumerate(self.output):
            if np.sum(edge.cars[:,0])==0: p[index]=1/edge.tto()
            else: p[index]=1/(np.sum(edge.cars[edge.cars[:,2]>0,2])/np.sum(edge.cars[:,0]))
        newcars=np.array([np.ones(N),np.zeros(N),np.random.normal(0,2,size=N),np.full(N,self.id),np.random.choice(range(len(self.output)),N,p=p/np.sum(p))]).transpose()
        newcars=np.concatenate((self.buildup,newcars)) #needs to handle the built up cars first
        self.buildup=np.zeros((0,5))
        for j,i in enumerate(self.output):
            going=newcars[:,4]==j
            filter = np.nonzero(i.cars[:,0]==0)[0]
            # print(np.sum(i.cars[:,0]==0),np.sum(going))
            if np.sum(i.cars[:,0]==0)>=np.sum(going): #can go if there are enough spaces for it
                # print("this case")
                i.cars[filter[:np.sum(going)],:]=newcars[going,:]
                i.cars[filter[:np.sum(going)],2]=np.ceil(i.tt()+i.cars[filter[:np.sum(going)],2])
                i.cars[filter[:np.sum(going)],4]=destinations[0]
                newcars=newcars[newcars[:,4]!=j]
            else: 
                spaces=np.sum(i.cars[:,0]==0)
                i.cars[filter,:]=newcars[np.nonzero(going==1)[0][:spaces],:]
                i.cars[filter,2]=np.ceil(i.tt()+i.cars[filter,2])
                i.cars[filter,4]=destinations[0]
                build=newcars[np.nonzero(going==1)[0][spaces:]] 
                # build[:,1]+=1 #update age of cars that stayed in the city
                self.buildup=np.concatenate((self.buildup,build)) #throw all excess cars into the buildup

In [802]:
# e_13 = edge(40000,2,id=13)
# e_a1 = edge(30000,2,id=1)
# n_a = node(output=[e_a1],id=4)
# n_1 = node(input=[e_a1],output=[e_13],id=1)
# n_3 = node(input=[e_13],id=3)

# for minute in range(240):
#     n_a.generate("",mean=85/2,sd=1.5/2,destinations=[3])
#     e_a1.update()
#     e_13.update()
#     n_1.flow()
#     n_3.flow()

In [803]:
# e_23 = edge(60000,2,id=23) 
# e_13 = edge(40000,2,id=13)
# e_a1 = edge(30000,2,id=1)
# e_a2 = edge(50000,2,id=2)
# n_a = node(output=[e_a1,e_a2],id=4)
# n_1 = node(input=[e_a1],output=[e_13],id=1)
# n_2 = node(input=[e_a2],output=[e_23],id=2)
# n_3 = node(input=[e_13,e_23],id=3)

# for minute in range(240):
#     n_a.generate("")
#     e_a1.update()
#     e_a2.update()
#     e_13.update()
#     e_23.update()
#     n_1.flow()
#     n_2.flow()
#     n_3.flow()

In [804]:
#importing the map
segments=[(1,2),(1,5),(5,2)]
seggy=0
alpha,beta=0.15,100
spits=True
ex13=0*10#%
extra=0*10 #%
alphabetatesting=True
saving=True
traffic_map = gpd.read_file("./map/2city_shape_v5.shp").to_crs(epsg=4326)
traffic_map['geometry'] = traffic_map['geometry'].scale(xfact= 1/100,yfact =1/100, origin = (0,0))
traffic_map.iloc[[7,9],4]=2*(100+extra)/100
traffic_map.iloc[13,4]=1*(100+ex13)/100
#capacity
traffic_map['capciteit'] = traffic_map['lengte']*traffic_map['banen']*1/(4.5+55)
# splitting_tabels

Node_df = pd.concat((traffic_map[traffic_map['Category'] == 'City'].iloc[:,:], traffic_map[traffic_map['Category'] == 'Node'].iloc[:,:])) 
Edge_df = traffic_map[traffic_map['Category'] == 'Edge'].iloc[:,:]

#printing the tables
print(traffic_map)
#print(Node_df)
#print(Edge_df)

    id     name   lengte  maxshd  banen Category  \
0    1    City1      NaN     NaN    NaN     City   
1    2    City2      NaN     NaN    NaN     City   
2    3   Node_A      NaN     NaN    NaN     Node   
3    4   Node_B      NaN     NaN    NaN     Node   
4    5   Node_C      NaN     NaN    NaN     Node   
5    6   Node_D      NaN     NaN    NaN     Node   
6    7   Node_E      NaN     NaN    NaN     Node   
7    8  Edge_1A  30000.0   100.0    2.0     Edge   
8    9  Edge_1B  50000.0   100.0    2.0     Edge   
9   10  Edge_AC  40000.0   100.0    2.0     Edge   
10  11  Edge_BC  60000.0   100.0    2.0     Edge   
11  12  Edge_CD  20000.0   100.0    2.0     Edge   
12  13  Edge_D2  10000.0   100.0    2.0     Edge   
13  14  Edge_C2  20000.0   100.0    1.0     Edge   
14  15  Edge_CE  30000.0   100.0    2.0     Edge   
15  16  Edge_E2   5000.0   100.0    2.0     Edge   

                                             geometry    capciteit  
0   POLYGON ((0.66646 -3.58556, 0.81725 -3.338

In [805]:
#making nodes and edges
nodedict=dict(zip(Node_df['id'],[node([],[],id=i) for i in Node_df['id']]))
edgedict=dict(zip(Node_df['id'],[dict() for i in Node_df['id']]))

for i in Edge_df["id"]:

    start,end=list(str(np.array(Edge_df['name'].loc[Edge_df['id']==i]))[-4:-2])
    for j in Node_df["name"]:
        if start in j: startid=str(np.array(Node_df['id'].loc[Node_df['name']==j])[0])
        if end in j: endid=str(np.array(Node_df['id'].loc[Node_df['name']==j])[0])
    edgedict[int(startid)][int(endid)]=edge(Edge_df['lengte'][i-1],Edge_df['banen'][i-1],id=i)

for i in nodedict.keys():
    nodedict[i].output=[edgedict[i][j]for j in edgedict[i].keys()]
    for j in edgedict[i].keys():
        nodedict[j].input.append(edgedict[i][j])
for i in range(7):
    nodedict[i+1].initwaitlist()
    # print(nodedict[i+1].input)

In [806]:
minutes1=120 #initialisation time
minutes2=120 #simulation time
tf_df = traffic_map.copy()
cars=np.zeros((len(tf_df["id"]),minutes2))

city_start,city_end=segments[seggy]
for minute in range(minutes1):
    if spits:nodedict[city_start].generate('',mean=95,sd=1,destinations=[city_end])
    else:nodedict[city_start].generate('',destinations=[city_end])
    for j in list(nodedict.keys())[:]:
        nodedict[j].flow()
    for j in edgedict.keys():
        for k in edgedict[j].keys(): 
            edgedict[j][k].update()
nodedict[city_end].times=np.array([]) #remove cars that arrived during simulation
    
for minute in range(minutes2):
    # print(minute)
    if spits:nodedict[city_start].generate('',mean=95,sd=1,destinations=[city_end])
    else:nodedict[city_start].generate('',destinations=[city_end])
    for j in list(nodedict.keys())[:]:
        nodedict[j].flow()
    print(edgedict[5][2].leftthismin)
    for j in edgedict.keys():
        for k in edgedict[j].keys(): 
            edgedict[j][k].update()
    for i in nodedict:
        cars[int(i)-1,minute]=nodedict[i].size()
    for i in Edge_df["id"]:
        start,end=list(str((Edge_df['name'][i-1]))[-2:])
        for index,j in enumerate(Node_df["name"]):
            if start in j: startid=(Node_df['id'][index])
            if end in j: endid=(Node_df['id'][index])
        cars[int(i)-1,minute]=np.sum(edgedict[int(startid)][int(endid)].cars[:,0])

#making a nice dataframe    
cars_time_df = pd.DataFrame(cars)


tf_df = pd.concat((tf_df,cars_time_df),axis = 1)
# print(tf_df)

27
18
26
32
25
23
26
28
23
20
26
29
23
34
22
16
25
23
29
24
37
21
28
30
20
21
22
27
35
19
30
24
19
25
23
28
26
26
26
25
26
24
35
29
25
15
21
28
24
23
26
23
31
25
22
26
31
34
17
19
21
27
28
26
21
21
22
33
19
33
20
24
33
28
31
20
20
16
20
28
41
24
27
26
25
18
20
32
22
26
21
24
29
31
22
22
31
25
24
25
27
21
21
28
27
24
27
27
26
23
29
23
17
29
19
24
25
26
23
31


In [807]:
print(nodedict[city_start].buildup.shape[0]/(minutes1+minutes2-50))

19.063157894736843


In [808]:
print(edgedict[5][7].tto()+edgedict[7][2].tto())
print(edgedict[1][3].tto()+edgedict[3][5].tto())
print(edgedict[1][4].tto()+edgedict[4][5].tto())

21.0
42.0
66.0


In [809]:
p=np.array([1/12.423427270701008,1/13.222122536974837 ,1/18.143516912347263])
print(p/np.sum(p))

[0.38105037 0.35803265 0.26091698]


In [810]:
for e,it in edgedict.items():
    for i,t in it.items():
        print(e,i,t.tt(),t.tto(),t.size(),t.C(),t.leftthismin_avg,t.id)

1 3 18.071111042552026 18.0 972.0 1008 38.47968926085681 8
1 4 30.0 30.0 1051.0 1680 36.30981980562187 9
3 5 24.32341423618572 24.0 1312.0 1344 37.83163740769836 10
4 5 36.0 36.0 1391.0 2016 36.40658951116102 11
5 6 12.0 12.0 386.0 672 29.470827138742035 12
5 2 13.799999999999999 12.0 336.0 336 25.05841791350335 14
5 7 18.0 18.0 353.0 1008 19.04105942850169 15
6 2 6.0 6.0 185.0 336 30.05174188647042 13
7 2 3.0 3.0 58.0 168 20.64391251948169 16


In [811]:
print(edgedict[5][2].tt())
print(edgedict[5][6].tt()+edgedict[6][2].tt())
print(edgedict[5][7].tt()+edgedict[7][2].tt())

13.799999999999999
18.0
21.0


In [812]:
e=edgedict[1][3]
e.C()*e.speed/e.length*1000/60/e.lanes

28.0

In [813]:
a=e.cars[:,1]+e.cars[:,2]
print(np.average(a[a>0]),np.std(a[a>0]))

21.282921810699587 1.9941262776390025


In [814]:
a

array([22., 25., 22., ..., 19., 22., 22.])

In [815]:
# bin_count = int((max(times)-min(times)))
# f, (ax1) = plt.subplots(1, figsize=(10,4), dpi=80)
# pdf, bins, _ = ax1.hist(times, bins=bin_count, density=True)
# centers = (bins[:-1] + bins[1:])/2
# ax1.set_title("Normalized travel times")
# ax1.set_ylabel("Probability")
# ax1.set_xlabel("Travel time (min)")
# ax1.legend()

In [816]:
print(edgedict[1][3].tt()+edgedict[3][5].tt())
print(edgedict[1][4].tt()+edgedict[4][5].tt())

42.394525278737746
66.0


In [817]:
traffic_map

,id,name,lengte,maxshd,banen,Category,geometry,capciteit
0,1,City1,NaN,NaN,NaN,City,"POLYGON ((0.66646 -3.58556, 0.81725 -3.33882, ...",NaN
1,2,City2,NaN,NaN,NaN,City,"POLYGON ((13.00588 -2.81356, 13.20662 -2.74107...",NaN
2,3,Node_A,NaN,NaN,NaN,Node,"POLYGON ((4.65275 -0.85074, 4.81724 -0.76988, ...",NaN
3,4,Node_B,NaN,NaN,NaN,Node,"POLYGON ((4.82561 -6.53287, 5.05702 -6.49384, ...",NaN
4,5,Node_C,NaN,NaN,NaN,Node,"POLYGON ((7.45199 -3.78381, 7.6137 -3.68623, 7...",NaN
5,6,Node_D,NaN,NaN,NaN,Node,"POLYGON ((10.24566 -0.99293, 10.33488 -0.90092...",NaN
6,7,Node_E,NaN,NaN,NaN,Node,"POLYGON ((10.26518 -6.73362, 10.40458 -6.62209...",NaN
7,8,Edge_1A,30000.0,100.0,2.0,Edge,"POLYGON ((3.3995 -3.11049, 4.30284 -2.22387, 4...",1008.403361
8,9,Edge_1B,50000.0,100.0,2.0,Edge,"POLYGON ((3.40507 -5.62814, 3.43574 -5.5891, 4...",1680.672269
9,10,Edge_AC,40000.0,100.0,2.0,Edge,"POLYGON ((5.61324 -2.07053, 5.64391 -2.03707, ...",1344.537815


In [818]:
# histogram check
spit="spits" if spits else ""
seg=f"half{seggy}" if seggy else ""
extrastr=f"{extra}" if extra and seggy!=2 else""
ex13str=f"_k{ex13}" if ex13 and seggy!=1 else""
# name=f"model_withoutdijkstr{seg}{spit}{extrastr}{ex13str}"
if alphabetatesting: name=f"albettesting_a{int(alpha*100)}_b{beta}"
else: name=f"notageincityspits{minutes1}_{minutes2}"
n_b = nodedict[city_end]
print(n_b.times)
print(max(n_b.times))
print(min(n_b.times))
plt.hist(n_b.times,bins=int((max(n_b.times)-min(n_b.times))))
if saving: plt.savefig(name)
plt.show()
print(name)
if saving: 
    with open(f"{name}times","wb") as f:
        np.save(f,n_b.times,allow_pickle=False)

[67. 68. 67. ... 90. 87. 93.]
102.0
54.0


albettesting_a15_b100


In [819]:
for i in np.nonzero(edgedict[3][5].cars[:,2]<0):
    if i not in nodedict[5].waitlist[edgedict[5][2]][edgedict[3][5]]: print(i)
edgedict[3][5].cars[nodedict[5].waitlist[edgedict[5][2]][edgedict[3][5]]]

[   5    8   11   16   18   23   29   31   33   36   45   46   51   53
   57   58   66   70   79   80   83   84  101  116  117  120  121  125
  127  129  131  137  138  139  148  155  162  164  173  174  183  185
  190  200  202  210  223  229  231  236  249  250  255  258  259  262
  270  272  279  281  290  295  298  302  306  308  317  320  324  331
  333  347  352  359  360  362  371  372  376  378  380  381  391  393
  406  407  415  419  421  423  424  436  451  455  456  465  474  476
  477  497  506  508  510  517  525  527  528  534  545  546  548  549
  552  557  560  563  567  569  573  588  595  608  613  617  629  635
  638  644  646  649  656  660  661  671  672  679  680  681  690  701
  704  709  711  716  728  733  735  744  751  757  760  763  766  779
  787  805  807  816  820  822  829  833  843  845  851  853  878  880
  899  902  906  913  918  922  924  927  928  929  936  942  947  955
  958  960  963  977  981  985  992  996  997 1004 1011 1013 1016 1036
 1039 

array([[  1.,  72., -20.,   1.,   2.],
       [  1.,  76., -20.,   1.,   2.],
       [  1.,  76., -20.,   1.,   2.],
       ...,
       [  1.,  56.,  -1.,   1.,   2.],
       [  1.,  56.,  -1.,   1.,   2.],
       [  1.,  57.,  -1.,   1.,   2.]])

In [820]:
nodedict[5].waitlist[edgedict[5][2]]

{edge 10 cars:1312.0: array([  79,  138,  236,  302,  308,  324,  958, 1159, 1173, 1270,    8,
          84,  117,  320,  333,  347,  415,  545,  569,  744, 1036,  162,
         190,  249,  259,  393,  477,  629,  660,  779,  880,  924, 1011,
        1078, 1113, 1151, 1244, 1272,   58,   83,  139,  173,  258,  270,
         372,  419,  845,  899,   70,  407,  563,  613,  671,  947,  960,
        1016, 1194,   16,  210,  250,  352,  359,  465,  534,  548,  595,
         704,  716,  807, 1057, 1072, 1105, 1110, 1311,  155,  183,  306,
         455,  680,  985, 1082, 1332,   57,   80,  137,  510,  728,  922,
         977, 1080, 1187, 1232,  371,  423,  527,  546,  617,  760,  853,
         936, 1039, 1040, 1064, 1119, 1314,    5,   23,  116,  164,  185,
         376,  378,  646,  679,  709,  820,  833,  963,  996, 1118, 1149,
        1277, 1323,   33,   46,   66,  121,  406,  476,  549,  608,  757,
         997, 1301,  101,  125,  127,  131,  290,  506,  508,  635,  735,
         766,  82

In [821]:
Raise ValueError

SyntaxError: invalid syntax (2153497262.py, line 1)

In [ ]:
# # histogram check
# print(n_3.times)
# print(max(n_3.times))
# print(min(n_3.times))
# plt.hist(n_3.times,bins=int((max(n_3.times)-min(n_3.times))))
# plt.savefig("onlytoppath")
# plt.show()

**Plotting**

In [ ]:
#plotting df

print(tf_df)



Plot_Node_df = pd.concat((tf_df[tf_df['Category'] == 'City'].iloc[:,:], tf_df[tf_df['Category'] == 'Node'].iloc[:,:])) 
Plot_Edge_df = tf_df[tf_df['Category'] == 'Edge'].iloc[:,:]
print(Plot_Node_df[100])

    id     name   lengte  maxshd  banen Category  \
0    1    City1      NaN     NaN    NaN     City   
1    2    City2      NaN     NaN    NaN     City   
2    3   Node_A      NaN     NaN    NaN     Node   
3    4   Node_B      NaN     NaN    NaN     Node   
4    5   Node_C      NaN     NaN    NaN     Node   
5    6   Node_D      NaN     NaN    NaN     Node   
6    7   Node_E      NaN     NaN    NaN     Node   
7    8  Edge_1A  30000.0   100.0    2.0     Edge   
8    9  Edge_1B  50000.0   100.0    2.0     Edge   
9   10  Edge_AC  40000.0   100.0    2.0     Edge   
10  11  Edge_BC  60000.0   100.0    2.0     Edge   
11  12  Edge_CD  20000.0   100.0    2.0     Edge   
12  13  Edge_D2  10000.0   100.0    2.0     Edge   
13  14  Edge_C2  20000.0   100.0    1.0     Edge   
14  15  Edge_CE  30000.0   100.0    2.0     Edge   
15  16  Edge_E2   5000.0   100.0    2.0     Edge   

                                             geometry    capciteit        0  \
0   POLYGON ((0.66646 -3.58556, 0.81

In [ ]:
#adding vehicles
data1= cars.copy()

cars_list = data1[:,0]


traffic_map.insert(3, "cars", cars_list, True)

#adding textprompts:




def gen_text(string):
    row_index = traffic_map.index[traffic_map['name'] == string]
    if "City" in string:
        return "City " + string[-1]
    elif 'Node' in string:
        if row_index!=None:
            num_of_car = traffic_map.loc[row_index[0], 'cars']
            return '\n' + string[-1] + f'\n cars: {num_of_car}'
        else:
            return '\n' + string[-1] + '\n cars: Not found'
    elif 'Edge' in string:
        if row_index!=None:
            num_of_car = traffic_map.loc[row_index[0], 'cars']
            return f'cars: {num_of_car}'
        else:
            return 'cars: Not found'
    else:
        return ''


traffic_map['textpromt'] = traffic_map['name'].apply(gen_text)




print(traffic_map)

    id     name   lengte     cars  maxshd  banen Category  \
0    1    City1      NaN  10198.0     NaN    NaN     City   
1    2    City2      NaN      0.0     NaN    NaN     City   
2    3   Node_A      NaN      0.0     NaN    NaN     Node   
3    4   Node_B      NaN      0.0     NaN    NaN     Node   
4    5   Node_C      NaN      0.0     NaN    NaN     Node   
5    6   Node_D      NaN      0.0     NaN    NaN     Node   
6    7   Node_E      NaN      0.0     NaN    NaN     Node   
7    8  Edge_1A  30000.0    973.0   100.0    2.0     Edge   
8    9  Edge_1B  50000.0    963.0   100.0    2.0     Edge   
9   10  Edge_AC  40000.0   1288.0   100.0    2.0     Edge   
10  11  Edge_BC  60000.0   1176.0   100.0    2.0     Edge   
11  12  Edge_CD  20000.0    368.0   100.0    2.0     Edge   
12  13  Edge_D2  10000.0    190.0   100.0    2.0     Edge   
13  14  Edge_C2  20000.0    336.0   100.0    1.0     Edge   
14  15  Edge_CE  30000.0    369.0   100.0    2.0     Edge   
15  16  Edge_E2   5000.0

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 8),dpi = 200)
cmap = plt.cm.get_cmap('Reds', 120)
cmap2 = plt.cm.get_cmap('Greens', 120)
print(cmap)
#cax = fig.add_axes([0.92, 0.1, 0.02, 0.78])  # Adjust the position and size as needed
data1[1] = 0
cax2 = fig.add_axes([0.06, 0.1, 0.02, 0.78])  # Adjust the position and size as needed
def update(i):
    ax.clear()
    traffic_map["cars"] = data1[:,i]
    traffic_map['textpromt'] = traffic_map['name'].apply(gen_text)
    Plot_Node_df.plot( cmap=cmap2, linewidth=0.4, ax=ax, edgecolor=".4",vmin=0,vmax=np.ceil(Plot_Node_df[i].max()/100)*100) ###hier moest er ook nog afgerond worden
    #plotting edges colored by fractions of capacity
    Plot_Edge_df.plot(column=Plot_Edge_df[i]/Plot_Edge_df["capciteit"], cmap=cmap, linewidth=0.4, ax=ax, edgecolor=".4",vmin=0,vmax=1)
    bar_info2 = plt.cm.ScalarMappable(cmap=cmap2, norm=plt.Normalize(vmin=0, vmax=np.ceil(Plot_Node_df[i].max()/100)*100))
    cbar = fig.colorbar(bar_info2, cax=cax2)
    
    #creating labels
    traffic_map.apply(lambda x: ax.annotate(text=x['textpromt'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1)
anim = FuncAnimation(fig,update,interval = 100)

# Create a colorbar
cax = fig.add_axes([0.92, 0.1, 0.02, 0.78])  # Adjust the position and size as needed
bar_info = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=100))
bar_info._A = [] ### geen idee wat dit doet
cbar = fig.colorbar(bar_info, cax=cax)

